## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests as req
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Marcos,BR,-28.9711,-51.0681,66.72,98,32,3.94,scattered clouds
1,1,Oxapampa,PE,-10.5775,-75.4017,59.67,98,97,2.55,overcast clouds
2,2,Chlorakas,CY,34.8042,32.4083,54.48,67,82,9.22,broken clouds
3,3,Grindavik,IS,63.8424,-22.4338,41.47,93,75,3.44,broken clouds
4,4,Clyde River,CA,70.4692,-68.5914,-1.97,84,40,10.36,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,La Palma,US,33.8464,-118.0467,83.97,55,20,8.05,few clouds
7,7,Cayenne,GF,4.9333,-52.3333,82.09,1,40,2.30,scattered clouds
12,12,Jizan,SA,17.3333,42.6667,83.79,49,0,3.02,clear sky
18,18,Hatillo,PR,18.4863,-66.8254,77.79,76,14,8.01,few clouds
20,20,Touros,BR,-5.1989,-35.4608,77.09,85,8,11.63,clear sky
24,24,Georgetown,MY,5.4112,100.3354,84.13,79,20,4.61,few clouds
30,30,Nizwa,OM,22.9333,57.5333,80.49,35,100,1.79,overcast clouds
40,40,Bubaque,GW,11.2833,-15.8333,78.31,74,6,7.72,clear sky
43,43,Hithadhoo,MV,-0.6000,73.0833,84.49,72,100,6.31,overcast clouds
47,47,Jalu,LY,29.0331,21.5482,76.24,19,99,15.28,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                149
City                   149
Country                149
Lat                    149
Lng                    149
Max Temp               149
Humidity               149
Cloudiness             149
Wind Speed             149
Current Description    149
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
updated_pcities_df = preferred_cities_df.dropna()
updated_pcities_df.count()

City_ID                149
City                   149
Country                149
Lat                    149
Lng                    149
Max Temp               149
Humidity               149
Cloudiness             149
Wind Speed             149
Current Description    149
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = updated_pcities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,La Palma,US,83.97,few clouds,33.8464,-118.0467,
7,Cayenne,GF,82.09,scattered clouds,4.9333,-52.3333,
12,Jizan,SA,83.79,clear sky,17.3333,42.6667,
18,Hatillo,PR,77.79,few clouds,18.4863,-66.8254,
20,Touros,BR,77.09,clear sky,-5.1989,-35.4608,
24,Georgetown,MY,84.13,few clouds,5.4112,100.3354,
30,Nizwa,OM,80.49,overcast clouds,22.9333,57.5333,
40,Bubaque,GW,78.31,clear sky,11.2833,-15.8333,
43,Hithadhoo,MV,84.49,overcast clouds,-0.6000,73.0833,
47,Jalu,LY,76.24,overcast clouds,29.0331,21.5482,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = req.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   137
Country                137
Max Temp               137
Current Description    137
Lat                    137
Lng                    137
Hotel Name             137
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))